In [1]:
import torch
import pandas as pd
import numpy as np
import math

from pathlib import Path
from typing import Dict, List
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from transformers import pipeline


2023-09-09 18:18:09.406077: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 18:18:09.430613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 18:18:09.824250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
notebook_login()

In [3]:
!python prepare_messages.py --tg-history-path "data/result.json" --output-path "data/data.csv"

/bin/bash: /home/consent-flower/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
data = load_dataset("csv", data_files="data/data.csv", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/consent-flower/anaconda3/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/consent-flower/.cache/huggingface/datasets/csv/default-fac9a14f9654e05d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [5]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 121829
})


In [6]:
data = data.filter(lambda example: example["context_1"] != None)
data

Filter:   0%|          | 0/121829 [00:00<?, ? examples/s]

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})

In [7]:
FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'
CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']
SEP = ' '


def convert_to_dialog(sample: Dict[str, str]) -> Dict[str, str]:
    result_dict = dict()
    dialog = ""
    for i in range(len(CONTEXT_COLS)):
        key = CONTEXT_COLS[i]
        if key in sample and sample[key] is not None:
            speaker_token = FIRST_SPEAKER_TOKEN if i % 2 == 0 else SECOND_SPEAKER_TOKEN
            dialog += speaker_token + SEP + sample[key] + SEP
    
    response_key = RESPONSE_COL[0]
    if response_key in sample and sample[response_key] is not None:
        dialog += SECOND_SPEAKER_TOKEN + SEP + sample[response_key]
    result_dict['text'] = dialog
    return result_dict

In [8]:
data = data.train_test_split(test_size=0.2, shuffle=True)

In [9]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# https://huggingface.co/docs/transformers/tasks/language_modeling

In [11]:
dialog_data = data.map(convert_to_dialog)

Map:   0%|          | 0/63284 [00:00<?, ? examples/s]

Map:   0%|          | 0/15821 [00:00<?, ? examples/s]

In [12]:
type(dialog_data["train"]["text"])

list

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=128)

In [14]:
tokenized_data = dialog_data.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=data["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/63284 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/15821 [00:00<?, ? examples/s]

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [17]:
training_args = TrainingArguments(
    output_dir="model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=32,
)

In [18]:
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-medium').to(device)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ninachely. Use `wandb login --relogin` to force relogin


  0%|          | 0/5934 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.214, 'learning_rate': 1.831479609032693e-05, 'epoch': 0.25}
{'loss': 1.0715, 'learning_rate': 1.662959218065386e-05, 'epoch': 0.51}
{'loss': 1.0471, 'learning_rate': 1.494438827098079e-05, 'epoch': 0.76}


  0%|          | 0/1978 [00:00<?, ?it/s]

...

  0%|          | 0/1978 [00:00<?, ?it/s]

{'eval_loss': 0.9772012829780579, 'eval_runtime': 104.0229, 'eval_samples_per_second': 152.092, 'eval_steps_per_second': 19.015, 'epoch': 2.0}
{'loss': 0.9596, 'learning_rate': 6.518368722615437e-06, 'epoch': 2.02}
{'loss': 0.941, 'learning_rate': 4.833164812942366e-06, 'epoch': 2.28}
{'loss': 0.9417, 'learning_rate': 3.147960903269296e-06, 'epoch': 2.53}
{'loss': 0.9378, 'learning_rate': 1.4627569935962252e-06, 'epoch': 2.78}


  0%|          | 0/1978 [00:00<?, ?it/s]

{'eval_loss': 0.9686253070831299, 'eval_runtime': 103.2475, 'eval_samples_per_second': 153.234, 'eval_steps_per_second': 19.158, 'epoch': 3.0}
{'train_runtime': 4020.5958, 'train_samples_per_second': 47.22, 'train_steps_per_second': 1.476, 'train_loss': 1.0014244930728904, 'epoch': 3.0}


TrainOutput(global_step=5934, training_loss=1.0014244930728904, metrics={'train_runtime': 4020.5958, 'train_samples_per_second': 47.22, 'train_steps_per_second': 1.476, 'train_loss': 1.0014244930728904, 'epoch': 3.0})

In [20]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/1978 [00:00<?, ?it/s]

Perplexity: 2.63


In [21]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

'https://huggingface.co/ninachely/model/tree/main/'

In [43]:
checkpoint_path = '/home/consent-flower/tinkoff/sirius-test-nlp-2023/model/checkpoint-5500'

# tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelWithLMHead.from_pretrained(checkpoint_path)

/home/consent-flower/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [154]:
def process_response(response):
    response = response.capitalize()
    response = response.replace("\n", " ")
    return response


def print_dialogue(prompt, response):
    print("— " + prompt)
    print("— " + response)

In [301]:
prompt = "Как попасть на завод?"

In [302]:
full_prompt = f'@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ {prompt} @@ВТОРОЙ@@'
inputs = tokenizer(full_prompt, return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=10,
    top_p=0.95,
    num_beams=3,
    num_return_sequences=3,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.2,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]

response = context_with_response[2][len(full_prompt)+1:]
response = response[:response.find("<pad>")]
response = process_response(response)

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


In [303]:
print_dialogue(prompt, response)

— Как попасть на завод?
— А зачем? ты что, хочешь стать инженером-конструктором?) или программистом?
